建立物品到用户 倒排表
表示某个物品，被哪几名用户访问过

In [1]:
import math

def UsersSimilarity(train_set):
    
    #建立物品到用户 倒排表
    
    item2user = dict()
    for user, items in train_set.items():
        for item in items:
            if item not in item2user:
                item2user[item] = []
            item2user[item].append(user)
            
    users_similarity_matrix = {}
    
    for items, users in item2user.items():
        for user_i in users:
            for user_j in users:
                #排除掉自己，i,i
                if user_i == user_j:
                    continue
                #default -- 键不存在时，设置的默认键值。
                users_similarity_matrix.setdefault(user_i, {})
                users_similarity_matrix[user_i].setdefault(user_j, 0)
                users_similarity_matrix[user_i][user_j] += 1
                #C[u][v] += 1 / math.log(1 + len(users)) 
    #user_sim {1:{2:6, 4:10} }  一号用户 与 2 号相似6 与 4号相似10      
    #。扫描完所有物品后，我们可以得到最终的W矩阵。
#这里的W是余弦相似度中的分子部分，然后将W除以分母可以得到最终的用户兴趣相似度。
    for user, related_users in users_similarity_matrix.items():
        for user_j, cuv in related_users.items():
            users_similarity_matrix[user_i][user_j] = cuv/math.sqrt(len(train_set[user] * len(train_set[user_j])))                                                                  
    return users_similarity_matrix                                                                                                                       

In [2]:
import dataloader
import os 

M = 5
data = dataloader.load_data(os.path.join(os.getcwd(),'Data','ratings.dat'))
train_set, test_set = dataloader.split_data(data, M, 100)

sliting data



In [3]:
print(train_set[1])
print('*********\n')
print(test_set[1])

['1193', '3408', '2355', '1287', '2804', '594', '919', '595', '2398', '2791', '2018', '2797', '1270', '527', '48', '1097', '1721', '1545', '2294', '3186', '1566', '588', '1907', '783', '1836', '1022', '2762', '150', '1', '1962', '260', '1028', '1029', '1207', '2028', '531', '3114', '1246']
*********

['938', '2918', '1035', '3105', '1961', '2692', '608']


In [4]:
user_sim_matrix = UsersSimilarity(train_set)
# takes a long time about half an hour

In [5]:
def save_dict2json(file_name, dic): 
    import json
    with open(file_name+'.json','a') as outfile:
        json.dump(dic,outfile,ensure_ascii=False)
        outfile.write('\n')

def read_json2dict(file_path):
    import json
    data = []
    with  open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    f.close()
    if len(data) ==  1:
        return data[0]
    else:
        return data

In [7]:
save_dict2json('user_sim_matrix', user_sim_matrix)

得到一个用户之间的相似度矩阵 N*N


In [8]:
#得到用户之间的兴趣相似度后，UserCF算法会给用户推荐和他兴趣最相似的K个用户喜欢的物品。
#这里选50个，选出50个与某用户最接近的K个用户。
similarity_user = 80

In [9]:
def Recommend(user, train_set, W, k = similarity_user):
    already_view_items = train_set[user] 
    rank = dict()
    for user_j, weight_user_ij in sorted(W[user].items(), key = lambda w_user_j: w_user_j[1],reverse=True)[:k]:
        for items in train_set[user_j]:
            if items in already_view_items:
                continue;
            else:
                rank.setdefault(items, 0)
                rank[items] += weight_user_ij
                # real is  += weight_user_ij * rvi
    return sorted(rank.items(), key = lambda w:w[1], reverse = True)


In [10]:
Recommend(1,train_set,user_sim_matrix, k = 5)[:10]

[('3034', 129),
 ('923', 129),
 ('2268', 129),
 ('3499', 129),
 ('608', 129),
 ('2329', 129),
 ('2336', 129),
 ('3147', 129),
 ('141', 129),
 ('1784', 129)]

In [59]:
def Recommend2(user, train, W,   k = similarity_user):
    rank = dict()
    from operator import itemgetter
    watched_items = train[user]
    for v, wuv in sorted(W[user].items(),key=itemgetter(1), reverse=True)[:k]:      #obtained most k user which similarity given user
        for movie in train[v]:
            if movie in watched_items:
                continue
            rank.setdefault(movie, 0)
            rank[movie] += wuv
    #return most n movies which have large probability given user will like these. Return type: [{movieID: similarity score}, {},...,{}]
    return sorted(rank.items(), key=itemgetter(1), reverse=True)

In [11]:
class  myEvalution():
    def __init__(self, train_set, test_set, users_sim_matrix, TopN = 10):
        self.train = train_set
        self.test = test_set
        self.N = TopN
        self.sim_mat = users_sim_matrix
        self.hit = 0
        self.all = 0
        self.recommend_items = set()
        self.all_items = set()
        
    def run(self):
        for users in train_set.keys():
            for item in self.train[users]:
                self.all_items.add(item)
                #add all different items 
            tu = self.test.get(users, {}) #获得测试集中的user 评分电影 如果没有返回空{}

            #计算user最感兴趣的前N个items
            Init_rank = Recommend(users, self.train, self.sim_mat, k = similarity_user)
            rank = Init_rank[:self.N]

            for item, weight in rank:
                if item in tu:
                    self.hit += 1
                    #计算 推荐和 测试集上的交集
                self.recommend_items.add(item)
            self.all += len(tu)
    
    def recall(self):
        #召回率， 对用户推荐的N个item 与 测试集 item 交集， 除以测试集中的数量
        #预测出来正确的个数 /  真正为正确的个数
        return self.hit / (self.all * 1.0) *100
    
    def precision(self):
        #预测出来正确的个数 / 预测的总个数
        return self.hit / ((len(self.train) * self.N * 1.0))*100
    
    def coverage(self):
        
        return len(self.recommend_items) / (len(self.all_items) * 1.0) *100
        

In [12]:
model = myEvalution(train_set, test_set, user_sim_matrix)
model.run()


In [89]:
print('recall is {:5f} % , precision is {:5f} % , coverage is {:5f} %'.format(model.recall(), model.precision(), model.coverage()))

recall is 5.230972 % , precision is 17.306291 % , coverage is 13.302503 %


recall is 5.230972 % , precision is 17.306291 % , coverage is 13.302503 %

In [64]:
# -*- coding: utf-8 -*-
import math
import random
import re
import json
from operator import itemgetter

REC_NUMBER = 10
similarity_user = 50

def SplitData(filename, M, seed):           #读取数据，分割为训练集和测试集：M表示 1/M 的样本为测试集
    test = dict()
    train = dict()                          #数据集以dict的方式存储，key为user id， value为user看过的movie id组成的list
    random.seed(seed)
    with open(filename,'r') as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            #user, movie, rating, timestamp = line.split(' ')
            #temp = re.split('\s+', line)
            temp = line.split('::')
            user = temp[0]; movie = temp[1]; rating = temp[2]
            user = int(user)
            if float(rating) < 4:                   #此处int(rating)会报错；4分以下的评分忽略
                continue
            if random.randint(1,M) == 1:
                if user not in test:
                    test[user] = []
                test[user].append(movie)
            else:
                if user not in train:
                    train[user] = []
                train[user].append(movie)
    return test, train



def UserSimilarity(train):
    #build inverse table for item_users: it's shown that each item was selected by which users
    item_users = {}
    for u, items in train.items():
        for i in items:
            if i not in item_users:
                item_users[i] = []
            item_users[i].append(u)

    user_sim_matrix = {}
    for i, users in item_users.items():
        for u in users:
            for v in users:
                if u == v:
                    continue
                user_sim_matrix.setdefault(u, {})
                user_sim_matrix[u].setdefault(v, 0)
                user_sim_matrix[u][v] += 1

    #calculate finial similarity matrix W
    for u, related_users in user_sim_matrix.items():
        for v, cuv in related_users.items():
            user_sim_matrix[u][v] = cuv / math.sqrt(len(train[u]) * len(train[v]))
    return user_sim_matrix

def Recommend(user, train, W):
    rank = dict()
    k = similarity_user
    watched_items = train[user]
    for v, wuv in sorted(W[user].items(),key=itemgetter(1), reverse=True)[:k]:      #obtained most k user which similarity given user
        for movie in train[v]:
            if movie in watched_items:
                continue
            rank.setdefault(movie, 0)
            rank[movie] += wuv
    #return most n movies which have large probability given user will like these. Return type: [{movieID: similarity score}, {},...,{}]
    return sorted(rank.items(), key=itemgetter(1), reverse=True)


class Evaluation():

    def __init__(self, train, test, W):
        self.train = train
        self.test = test
        self.W = W
        self.N = REC_NUMBER
        self.hit = 0
        self.hit_filter = 0
        self.all = 0
        self.recommend_items = set()
        self.recommend_items_filter = set()
        self.all_items = set()
#         readfile = '.'#'./Result//11.19/1'
#         with open(readfile + '/Pmatrix.json', 'r') as f:
#             self.pmatrix = json.loads(f.read())
#         with open(readfile + '/Qmatrix.json', 'r') as f:
#             self.qmatrix = json.loads(f.read())

    def run(self):
        for user in self.train.keys():
            for item in self.train[user]:
                self.all_items.add(item)
            tu = self.test.get(user, {})  # user corresponding movie list in the test.
            Init_rank = Recommend(user, self.train, self.W)
            rank = Init_rank[:self.N]
            #Filter_rank = self.Filter(user, Init_rank)[:self.N]
            for item, _ in rank:
                if item in tu:
                    self.hit += 1
                self.recommend_items.add(item)
            self.all += len(tu)

#             for item, _ in Filter_rank:
#                 if item in tu:
#                     self.hit_filter += 1
#                 self.recommend_items_filter.add(item)

#     def Filter(self, user, InitRank):
#         user = str(user)
#         user_att = sorted(self.pmatrix[user].items(), key=itemgetter(1))
#         NegativeAtt = []
#         for i in range(10):
#             negative, _ = user_att[i]
#             NegativeAtt.append(negative)
#         WillRm = []
#         for item in self.qmatrix.keys():
#             item_att = sorted(self.qmatrix[item].items(), key=itemgetter(1), reverse=True)[:5]
#             for att, _ in item_att:
#                 if att in NegativeAtt:
#                     WillRm.append(item)
#                     break
#         for item, scoring in InitRank:
#             for will_rm in WillRm:
#                 if item == will_rm:
#                     InitRank.remove((item, scoring))
#         return InitRank

    def Recall(self):
        print('Recall: ', self.hit / (self.all * 1.0))
#         print('After filtering: ', self.hit_filter / (self.all * 1.0))

    def Precision(self):
        print('Precision: ', self.hit / (len(self.train) * self.N * 1.0))
#         print('After filtering: ', self.hit_filter / (len(self.train) * self.N * 1.0))

    def Coverage(self):
        print('Coverage: ', len(self.recommend_items) / (len(self.all_items) * 1.0))
#         print('After filtering: ', len(self.recommend_items_filter) / (len(self.all_items) * 1.0))


if __name__ == '__main__':
    filename = './Data/ratings.dat'
    # filename = './ml-100k/u.data'
    test, train = SplitData(filename, 5, random.random())
    W = UserSimilarity(train)
    result = Evaluation(train, test, W)
    result.run()
    result.Precision()
    print('\n')
    result.Recall()
    print('\n')
    result.Coverage()

Precision:  0.22892164982607255


Recall:  0.11968165718393044


Coverage:  0.17078071182548796


In [65]:
filename = './Data/ratings.dat'
# filename = './ml-100k/u.data'
test, train = SplitData(filename, 5, random.random())
W = UserSimilarity(train)
result = Evaluation(train, test, W)
result.run()
result.Precision()
print('\n')
result.Recall()
print('\n')
result.Coverage()

Precision:  0.23074374689415272


Recall:  0.12057474249112785


Coverage:  0.17081545064377682


In [75]:
result = Evaluation(train, test, user_sim_matrix)
result.run()
result.Precision()
print('\n')
result.Recall()
print('\n')
result.Coverage()

Precision:  0.1469272817624648


Recall:  0.07677659482385528


Coverage:  0.0709585121602289
